In [41]:
# mac m1
import pandas as pd
import openpyxl
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
plt.rcParams["axes.unicode_minus"] = False 
rc("font", family="Arial Unicode MS") # Windows: Malgun Gothic 
# %matplotlib inline 
get_ipython().run_line_magic("matplotlib", "inline")
from tqdm import tqdm
import folium

---

### 데이터 가공

In [42]:
df_bus = pd.read_csv('사용년월별_버스정류장_근처지하철_출퇴근_총이동량.csv',index_col=0)
df_bus

,사용년월,표준버스정류장ID,역명,line,name,lat,lng,총 이동량,출근,퇴근
0,201801,100000001,종로2가사거리,01호선,종로3가,37.571607,126.991806,3719,2686,1033
1,201801,100000002,창경궁.서울대학교병원,04호선,혜화,37.582336,127.001844,49548,33805,15743
2,201801,100000003,명륜3가.성대입구,04호선,혜화,37.582336,127.001844,61196,31984,29212
3,201801,100000004,종로2가.삼일교,01호선,종각,37.570161,126.982923,7776,3752,4024
4,201801,100000005,혜화동로터리,04호선,혜화,37.582336,127.001844,32691,23234,9457
...,...,...,...,...,...,...,...,...,...,...
100824,202112,225000155,산본1동입구,01호선,금정,37.372221,126.943429,2690,1011,1679
100825,202112,225000156,산본1동,01호선,금정,37.372221,126.943429,6474,4750,1724
100826,202112,225000157,산본고가도로,01호선,금정,37.372221,126.943429,3504,2128,1376
100827,202112,225000250,산본주유소,01호선,금정,37.372221,126.943429,1409,1208,201


In [43]:
df_bus_pivot = df_bus.pivot_table(index=['사용년월','name','lat','lng'],values=['출근','퇴근','총 이동량'],aggfunc=sum).reset_index()
df_bus_pivot

,사용년월,name,lat,lng,총 이동량,출근,퇴근
0,201801,가산디지털단지,37.481072,126.882343,458727,225301,233426
1,201801,가양,37.561391,126.854456,247439,116471,130968
2,201801,강남,37.497175,127.027926,681804,300452,381352
3,201801,강남구청,37.517186,127.041280,171488,84521,86967
4,201801,강동,37.535804,127.132481,95472,52396,43076
...,...,...,...,...,...,...,...
6055,202112,합정,37.549463,126.913739,185282,66787,118495
6056,202112,혜화,37.582336,127.001844,298638,148659,149979
6057,202112,홍대입구,37.557192,126.925381,517418,221436,295982
6058,202112,회기,37.589977,127.058031,263227,143730,119497


In [44]:
df_bus_pivot = df_bus_pivot.astype({'사용년월':'string'})
df_bus_pivot.dtypes

사용년월      string
name      object
lat      float64
lng      float64
총 이동량      int64
출근         int64
퇴근         int64
dtype: object

In [45]:
df_18 = df_bus_pivot[df_bus_pivot['사용년월'].str.contains('2018')]
df_19 = df_bus_pivot[df_bus_pivot['사용년월'].str.contains('2019')]
df_20 = df_bus_pivot[df_bus_pivot['사용년월'].str.contains('2020')]
df_21 = df_bus_pivot[df_bus_pivot['사용년월'].str.contains('2021')]

In [46]:
df_18_19 = pd.concat([df_18,df_19])
df_20_21 = pd.concat([df_20,df_21])

df_년도 : 해당 년도의 사용년월(1~12월)	name(가까운 지하철역 이름)	총 이동량	출근	퇴근    정보를 포함하는 데이터 프레임

In [47]:
df_18_19_pivot = df_18_19.pivot_table(index=['name','lat','lng'], aggfunc=sum).reset_index()
df_18_19_pivot.sort_values(['총 이동량','출근','퇴근'], ascending=False, inplace=True)
df_18_19_pivot

,name,lat,lng,총 이동량,출근,퇴근
69,신림,37.484201,126.929715,24519233,12508742,12010491
13,구로디지털단지,37.485266,126.901401,23855862,12658696,11197166
98,잠실,37.513950,127.102234,18067160,8450091,9617069
53,서울대입구,37.481247,126.952739,16747475,8079336,8668139
2,강남,37.497175,127.027926,15848960,6930370,8918590
...,...,...,...,...,...,...
41,미금,37.350077,127.108910,99155,29376,69779
103,정자,37.367060,127.108105,93683,33423,60260
38,모란,37.432130,127.129087,74693,23425,51268
119,평촌,37.394287,126.963883,34013,13364,20649


In [48]:
df_18_19_pivot.describe()

,lat,lng,총 이동량,출근,퇴근
count,129.000000,129.000000,1.290000e+02,1.290000e+02,1.290000e+02
mean,37.525127,126.995911,6.031379e+06,2.884616e+06,3.146763e+06
std,0.061237,0.074710,4.651175e+06,2.261289e+06,2.428214e+06
min,37.350077,126.753664,1.278100e+04,5.805000e+03,6.976000e+03
25%,37.491897,126.950806,2.768747e+06,1.357679e+06,1.455587e+06
50%,37.527098,127.001849,5.451609e+06,2.597404e+06,2.794296e+06
75%,37.564718,127.046835,7.553889e+06,3.745686e+06,4.222502e+06
max,37.738899,127.132481,2.451923e+07,1.265870e+07,1.201049e+07


In [49]:
df_20_21_pivot = df_20_21.pivot_table(index=['name','lat','lng'], aggfunc=sum).reset_index()
df_20_21_pivot.sort_values(['총 이동량','출근','퇴근'], ascending=False, inplace=True)
df_20_21_pivot
df_20_21_pivot.describe()

,lat,lng,총 이동량,출근,퇴근
count,129.000000,129.000000,1.290000e+02,1.290000e+02,1.290000e+02
mean,37.525127,126.995911,4.530038e+06,2.191543e+06,2.338495e+06
std,0.061237,0.074710,3.464180e+06,1.709152e+06,1.781332e+06
min,37.350077,126.753664,3.096600e+04,1.286100e+04,1.810500e+04
25%,37.491897,126.950806,2.172380e+06,1.054739e+06,1.002059e+06
50%,37.527098,127.001849,4.019064e+06,1.931243e+06,2.038044e+06
75%,37.564718,127.046835,5.629667e+06,2.908428e+06,3.036760e+06
max,37.738899,127.132481,1.824461e+07,9.577347e+06,8.720626e+06


In [50]:
df_20_21_pivot.loc[0]['lat']

37.481072

---

### 지도 시각화

In [51]:
bus_map = folium.Map(location=[37.5502, 126.982], zoom_start=12)

In [52]:
for i in range(len(df_18_19_pivot)):
    folium.Circle(location=(df_18_19_pivot.loc[i]['lat'],df_18_19_pivot.loc[i]['lng']),
                    color='white',
                    radius=2).add_to(bus_map)

    num = df_18_19_pivot.loc[i]['총 이동량']/250000

    folium.CircleMarker(location=(df_18_19_pivot.loc[i]['lat'],df_18_19_pivot.loc[i]['lng']),
                        radius=num,
                        fill=True,
                        color='blue',
                        popup=folium.Popup(df_18_19_pivot.loc[i]['name'], max_width=100)).add_to(bus_map)

In [53]:
for i in range(len(df_20_21_pivot)):
    folium.Circle(location=(df_20_21_pivot.loc[i]['lat'],df_20_21_pivot.loc[i]['lng']),
                    color='white',
                    radius=2).add_to(bus_map)

    num = df_20_21_pivot.loc[i]['총 이동량']/250000

    folium.CircleMarker(location=(df_20_21_pivot.loc[i]['lat'],df_20_21_pivot.loc[i]['lng']),
                        radius=num,
                        fill=True,
                        color='red',
                        popup=folium.Popup(df_18_19_pivot.loc[i]['name'], max_width=100)).add_to(bus_map)

In [54]:
bus_map

In [55]:
bus_map.save('버스 코로나 전 후 지도 시각화.html')

---

In [56]:
df_18

,사용년월,name,lat,lng,총 이동량,출근,퇴근
0,201801,가산디지털단지,37.481072,126.882343,458727,225301,233426
1,201801,가양,37.561391,126.854456,247439,116471,130968
2,201801,강남,37.497175,127.027926,681804,300452,381352
3,201801,강남구청,37.517186,127.041280,171488,84521,86967
4,201801,강동,37.535804,127.132481,95472,52396,43076
...,...,...,...,...,...,...,...
1531,201812,합정,37.549463,126.913739,216949,86500,130449
1532,201812,혜화,37.582336,127.001844,382757,179624,203133
1533,201812,홍대입구,37.557192,126.925381,612475,282443,330032
1534,201812,회기,37.589977,127.058031,309627,164861,144766
